In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, SVR
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score


In [2]:
df=pd.read_csv('AMD_indicators.csv')
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV,RSI,10 DAYS EMA,20 DAYS EMA,MACD
0,1980-03-17,0.000000,3.302083,3.125000,3.145833,3.145833,219600,0.177083,3.145833,0.000000,0.000000,0.000000,0.000000,NaN,3.145833,3.145833,0.000000
1,1980-03-18,0.000000,3.125000,2.937500,3.031250,3.031250,727200,0.187500,3.031250,0.000000,0.000000,0.000000,0.000000,NaN,3.125000,3.134921,-0.009141
2,1980-03-19,0.000000,3.083333,3.020833,3.041667,3.041667,295200,0.062500,3.041667,0.000000,0.000000,0.000000,0.000000,NaN,3.109848,3.126039,-0.015367
3,1980-03-20,0.000000,3.062500,3.010417,3.010417,3.010417,159600,0.052083,3.010417,0.000000,0.000000,0.000000,0.000000,NaN,3.091770,3.115028,-0.022563
4,1980-03-21,0.000000,3.020833,2.906250,2.916667,2.916667,130800,0.114583,2.916667,0.000000,0.000000,0.000000,0.000000,NaN,3.059933,3.096136,-0.035422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,2020-11-05,83.269997,83.500000,81.849998,83.000000,83.000000,46542300,1.650002,-0.269997,77.317142,79.339999,81.141428,3.412531,46.361410,79.425322,80.022547,-0.681415
10250,2020-11-06,83.519997,86.089996,82.669998,85.879997,85.879997,53829900,3.419998,2.360001,77.905714,79.327856,80.965714,3.921513,49.673958,80.598899,80.580399,-0.124774
10251,2020-11-09,84.239998,87.050003,82.769997,83.120003,83.120003,58580700,4.280006,-1.119995,79.259999,79.604999,80.935713,4.664679,56.983435,81.057282,80.822266,0.092593
10252,2020-11-10,81.930000,82.129997,77.629997,77.989998,77.989998,67006400,4.500000,-3.940002,79.988571,79.716428,80.936666,4.832765,52.591371,80.499594,80.552527,-0.147392


In [3]:
# df['H-L']=df['High']-df['Low']
# df['O-C']=df['Close']-df['Open']

In [4]:
def MA(dataset,days):
    n=days
    dataset['{} DAYS MA'.format(n)]=0
    for i in range(len(dataset)):
        dataset['{} DAYS MA'.format(n)].iloc[i]=sum(dataset['Close'].iloc[i-n:i])/n

In [5]:
# MA(df,7)
# MA(df,14)
# MA(df,21)

In [6]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV,RSI,10 DAYS EMA,20 DAYS EMA,MACD
0,1980-03-17,0.000000,3.302083,3.125000,3.145833,3.145833,219600,0.177083,3.145833,0.000000,0.000000,0.000000,0.000000,NaN,3.145833,3.145833,0.000000
1,1980-03-18,0.000000,3.125000,2.937500,3.031250,3.031250,727200,0.187500,3.031250,0.000000,0.000000,0.000000,0.000000,NaN,3.125000,3.134921,-0.009141
2,1980-03-19,0.000000,3.083333,3.020833,3.041667,3.041667,295200,0.062500,3.041667,0.000000,0.000000,0.000000,0.000000,NaN,3.109848,3.126039,-0.015367
3,1980-03-20,0.000000,3.062500,3.010417,3.010417,3.010417,159600,0.052083,3.010417,0.000000,0.000000,0.000000,0.000000,NaN,3.091770,3.115028,-0.022563
4,1980-03-21,0.000000,3.020833,2.906250,2.916667,2.916667,130800,0.114583,2.916667,0.000000,0.000000,0.000000,0.000000,NaN,3.059933,3.096136,-0.035422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,2020-11-05,83.269997,83.500000,81.849998,83.000000,83.000000,46542300,1.650002,-0.269997,77.317142,79.339999,81.141428,3.412531,46.361410,79.425322,80.022547,-0.681415
10250,2020-11-06,83.519997,86.089996,82.669998,85.879997,85.879997,53829900,3.419998,2.360001,77.905714,79.327856,80.965714,3.921513,49.673958,80.598899,80.580399,-0.124774
10251,2020-11-09,84.239998,87.050003,82.769997,83.120003,83.120003,58580700,4.280006,-1.119995,79.259999,79.604999,80.935713,4.664679,56.983435,81.057282,80.822266,0.092593
10252,2020-11-10,81.930000,82.129997,77.629997,77.989998,77.989998,67006400,4.500000,-3.940002,79.988571,79.716428,80.936666,4.832765,52.591371,80.499594,80.552527,-0.147392


In [7]:
import math
def std_dev(dataset,days):
    n=days
    dataset['{} DAYS STD DEV'.format(days)]=0
    dataset['sq(C-MA)']=(dataset['Close']-dataset['{} DAYS MA'.format(n)])**2
    for i in range(len(dataset)):
        dataset['{} DAYS STD DEV'.format(n)].iloc[i]=math.sqrt(sum(dataset['sq(C-MA)'].iloc[i-n:i])/n)
    dataset.drop('sq(C-MA)', axis=1,inplace=True)

In [8]:
# std_dev(df,7)

In [9]:
# df

In [10]:
# preprocessing : price rise (1) or fall (0) label
def add_label(df, column):
    label = []
    for i in range(len(df)):
        if(i == 0):
            label.append(0)
            continue
        if(df[column][i] >= df[column][i - 1]):
            label.append(1)
        else:
            label.append(0)
    return pd.Series(label)
    

df["label"] = add_label(df, "Close")
df

,Date,Open,High,Low,Close,Adj Close,Volume,H-L,O-C,7 DAYS MA,14 DAYS MA,21 DAYS MA,7 DAYS STD DEV,RSI,10 DAYS EMA,20 DAYS EMA,MACD,label
0,1980-03-17,0.000000,3.302083,3.125000,3.145833,3.145833,219600,0.177083,3.145833,0.000000,0.000000,0.000000,0.000000,NaN,3.145833,3.145833,0.000000,0
1,1980-03-18,0.000000,3.125000,2.937500,3.031250,3.031250,727200,0.187500,3.031250,0.000000,0.000000,0.000000,0.000000,NaN,3.125000,3.134921,-0.009141,0
2,1980-03-19,0.000000,3.083333,3.020833,3.041667,3.041667,295200,0.062500,3.041667,0.000000,0.000000,0.000000,0.000000,NaN,3.109848,3.126039,-0.015367,1
3,1980-03-20,0.000000,3.062500,3.010417,3.010417,3.010417,159600,0.052083,3.010417,0.000000,0.000000,0.000000,0.000000,NaN,3.091770,3.115028,-0.022563,0
4,1980-03-21,0.000000,3.020833,2.906250,2.916667,2.916667,130800,0.114583,2.916667,0.000000,0.000000,0.000000,0.000000,NaN,3.059933,3.096136,-0.035422,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10249,2020-11-05,83.269997,83.500000,81.849998,83.000000,83.000000,46542300,1.650002,-0.269997,77.317142,79.339999,81.141428,3.412531,46.361410,79.425322,80.022547,-0.681415,1
10250,2020-11-06,83.519997,86.089996,82.669998,85.879997,85.879997,53829900,3.419998,2.360001,77.905714,79.327856,80.965714,3.921513,49.673958,80.598899,80.580399,-0.124774,1
10251,2020-11-09,84.239998,87.050003,82.769997,83.120003,83.120003,58580700,4.280006,-1.119995,79.259999,79.604999,80.935713,4.664679,56.983435,81.057282,80.822266,0.092593,0
10252,2020-11-10,81.930000,82.129997,77.629997,77.989998,77.989998,67006400,4.500000,-3.940002,79.988571,79.716428,80.936666,4.832765,52.591371,80.499594,80.552527,-0.147392,0


In [11]:
#finding index
start=df.loc[df['Date']=='2009-04-06']
end=df.loc[df['Date']=='2019-04-05']
split=df.loc[df['Date']=='2017-04-03']
print(start.index,end.index,split.index)

Int64Index([7331], dtype='int64') Int64Index([9848], dtype='int64') Int64Index([9343], dtype='int64')


In [12]:
# X = df.drop(['label','Date'], axis=1)
# y=df['label']
# print("Dimension of the Target: ", y.shape)
# print("Dimension of the Data Matrix X: ", X.shape)

X = df.drop(['label','Date','Open','High','Low','Adj Close','O-C','7 DAYS MA','14 DAYS MA','21 DAYS MA','10 DAYS EMA','20 DAYS EMA'], axis=1)
y=df['label']
print("Dimension of the Target: ", y.shape)
print("Dimension of the Data Matrix X: ", X.shape)

Dimension of the Target:  (10254,)
Dimension of the Data Matrix X:  (10254, 6)


In [13]:
start_time, split_time, end_time=7331, 9343,9848
X_train,X_val=X.loc[start_time:split_time],X.loc[split_time:end_time]
y_train,y_val=y.loc[start_time:split_time],y.loc[split_time:end_time]
# start_time, split_time, end_time=9000, 9010,9015
# X_train,X_val=X.loc[start_time:split_time],X.loc[split_time:end_time]
# y_train,y_val=y.loc[start_time:split_time],y.loc[split_time:end_time]

In [14]:
X_train

,Close,Volume,H-L,7 DAYS STD DEV,RSI,MACD
7331,3.53,15234800,0.310000,0.282138,68.060838,0.270162
7332,3.44,11663000,0.100000,0.195693,65.714287,0.272174
7333,3.68,16869600,0.300000,0.174917,60.085839,0.289793
7334,3.75,19026000,0.240000,0.235014,64.062502,0.305880
7335,3.61,16038600,0.240000,0.280540,73.076925,0.303829
...,...,...,...,...,...,...
9339,13.69,37420200,0.309999,0.278700,57.756558,0.180800
9340,13.71,37863500,0.240000,0.263688,55.831256,0.160635
9341,14.05,44132800,0.400001,0.160889,54.822335,0.170129
9342,14.55,84566200,0.740000,0.180050,51.891896,0.215513


In [15]:

# print('开始训练特征')
# svc = SVC(kernel="linear", C=1)
# rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
# rfe.fit(X_train,y_train)
# print('训练特征选择完成，输出特征：')
# result = features.columns[rfe.get_support()]
# print(result)
# print(rfe.ranking_)
# estimator1 = SVR(kernel='linear')
# re = RFECV(estimator=estimator1, step=1, cv=10)
# re.fit(X_train,y_train)
# print('特征输出完毕')

In [25]:
# train SVM model
svm = SVC()
svm.fit(X_train, y_train)


pred_y_train = svm.predict(X_train)
print("accuracy :", end=" ")
print(str(round(metrics.accuracy_score(pred_y_train, y_train) * 100, 2)) + " %" )

print(y_train)
print(pred_y_train)
print(type(pred_y_train))


cnt = 0
for i in pred_y_train:
    if(i==0):
        cnt+=1
print(cnt)

pred_y = svm.predict(X_val)
print("accuracy :", end=" ")
print(str(round(metrics.accuracy_score(pred_y, y_val) * 100, 2)) + " %" )

accuracy : 52.91 %
7331    1
7332    0
7333    1
7334    1
7335    0
       ..
9339    0
9340    1
9341    1
9342    1
9343    1
Name: label, Length: 2013, dtype: int64
[1 1 1 ... 1 1 1]
<class 'numpy.ndarray'>
11
accuracy : 51.38 %


In [31]:
%%time

param_grid = {'gamma': [0.000001, 0.0001, 0.001, 0.01, 0.1, 1], 
              'C': [10, 100, 1000, 10000]}
clf = SVC()

clf_cv = GridSearchCV(clf, param_grid, scoring='accuracy', cv=5)
clf_cv.fit(X_train, y_train)

params_optimal = clf_cv.best_params_

print("Best Score (accuracy): %f" % clf_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal)
print("\n")

KeyboardInterrupt: 

In [30]:
svm_rbf_clf = SVC(kernel="rbf", **params_optimal)
svm_rbf_clf.fit(X_train, y_train)

y_train_predicted = svm_rbf_clf.predict(X_train)


print(y_train)
print(y_train_predicted)



cnt = 0
for i in y_train_predicted:
    if(i==0):
        cnt+=1
print(cnt)


# plt.figure(figsize=(12, 6))
# decision_boundary_svc_class_colored(svm_rbf_clf, X, plotDistanceFromHyperplane=True, colorBar=True)
# plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=50, cmap='autumn')
# plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, s=90, cmap='autumn', alpha=1.0, edgecolors="k")
# plt.title("Experiment 1: $\\gamma$ = Very Large (Overfitting)", fontsize=16)
# plt.axis([-2.5, 2.5, -2.5, 2.5])
# plt.show()

print("Training Accuracy: ", svm_rbf_clf.score(X_train, y_train))

y_val_predicted = svm_rbf_clf.predict(X_val)
print("Test Accuracy: ", svm_rbf_clf.score(X_val, y_val))


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_val, y_val_predicted))

precision_test = precision_score(y_val, y_val_predicted) 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_val, y_val_predicted)
print("Test Recall = %f" % recall_test)

f1_test = f1_score(y_val, y_val_predicted)
print("Test F1 Score = %f" % f1_test)

print("\nClassification Report:")
print(classification_report(y_val, y_val_predicted))

7331    1
7332    0
7333    1
7334    1
7335    0
       ..
9339    0
9340    1
9341    1
9342    1
9343    1
Name: label, Length: 2013, dtype: int64
[1 0 1 ... 1 1 1]
952
Training Accuracy:  0.9995032290114257
Test Accuracy:  0.5276679841897233

Test Confusion Matrix:
[[  0 235]
 [  4 267]]

Test Precision = 0.531873
Test Recall = 0.985240
Test F1 Score = 0.690815

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       235
           1       0.53      0.99      0.69       271

    accuracy                           0.53       506
   macro avg       0.27      0.49      0.35       506
weighted avg       0.28      0.53      0.37       506

